In [1]:
import os
import sys

root = os.path.dirname(os.path.dirname(os.path.dirname(os.getcwd())))
sys.path.append(root)

root

'/home/vtnthereal/working_dir/AsynFL'

In [2]:
from asynfed.client.frameworks.tensorflow import TensorflowFramework

from experiment.ember_dataset.ember_model import EmberModel
from experiment.ember_dataset.data_preprocessing import DataLoader



2023-08-13 14:49:43.124291: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-13 14:49:43.349591: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-13 14:49:44.428347: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [7]:
config = {'client_id': 'centralized-training',
 'role': 'trainer',
 'gpu_index': 0,
 'tracking_point': 10000,
 'dataset': {'chunk_index': 1, 'qod': 0.85},
 'training_params': {'epoch': 200,
  'batch_size': 128,
  'regularization': 'l2',
  'lambda_value': 0.0005,
  'learning_rate': 0.1,
  'beta': 0.6,
  'decay_period': 200}}

In [8]:
# train_data_loader = DataLoader("./../ember_data/chunk_0.pickle")
train_data_loader = DataLoader("chunk_1.pickle")
data_size = train_data_loader.get_dataset_size()
class_weight = train_data_loader.get_class_weight()
num_input_features = train_data_loader.get_num_input_features()
print(data_size, num_input_features, class_weight)

train_ds = train_data_loader.create_tensorflow_dataset()







60176 2381 [2.4280180761781796, 1.7002712477396023]


In [9]:
epoch = 10
batch_size = 128

learning_rate_config = config.get('training_params').get('learning_rate_config', {})
if learning_rate_config == {}:
    learning_rate_config['fix_lr'] = True
    learning_rate_config['initial_lr'] = 0.01
    learning_rate_config['min_lr'] = 0.001
    learning_rate_config['decay_steps'] = data_size * epoch / batch_size

In [10]:
# Define model
model = EmberModel(input_features= num_input_features, output_features= 1, lr_config= learning_rate_config, class_weight= class_weight)


lr config in the resnet model
{'fix_lr': True, 'initial_lr': 0.01, 'min_lr': 0.001, 'decay_steps': 4701.25}
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 2381)]       0           []                               
                                                                                                  
 embedding (Embedding)          (None, 2381, 8)      2056        ['input_1[0][0]']                
                                                                                                  
 conv1d (Conv1D)                (None, 158, 128)     15488       ['embedding[0][0]']              
                                                                                                  
 conv1d_1 (Conv1D)              (None, 158, 128)     15488       ['embedding[0][0]'] 

In [11]:
# Define framework
tensorflow_framework = TensorflowFramework(model=model,
                                           data_size= data_size,
                                           train_ds= train_ds,
                                           test_ds= None,
                                           config=config)


In [12]:
epoch = 10
test_acc = None
test_loss = None
for epoch in range(epoch):
    print(f"Enter epoch {epoch + 1}, learning rate = {tensorflow_framework.get_learning_rate()}")
    tensorflow_framework.model.train_loss.reset_states()
    tensorflow_framework.model.train_performance.reset_states()
    tensorflow_framework.model.test_loss.reset_states()
    tensorflow_framework.model.test_performance.reset_states()

    for images, labels in tensorflow_framework.train_ds:
        train_acc, train_loss= tensorflow_framework.fit(images, labels)

    print(f"Epoch {epoch+1} - Train Acc: {(train_acc * 100):.2f} -- Train Loss: {train_loss}")




Enter epoch 1, learning rate = 0.009999999776482582


2023-08-13 14:50:22.918511: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-08-13 14:50:26.272604: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8600
2023-08-13 14:50:28.391307: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-08-13 14:50:29.396885: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x7f383c2f2050 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-08-13 14:50:29.396948: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): NVIDIA GeForce RTX 3060 Laptop GPU, Compute Capability 8.6
2023-08-13 14:

Epoch 1 - Train Acc: 80.55 -- Train Loss: 1.0408155918121338
Enter epoch 2, learning rate = 0.009999999776482582
Epoch 2 - Train Acc: 92.41 -- Train Loss: 0.499462366104126
Enter epoch 3, learning rate = 0.009999999776482582
Epoch 3 - Train Acc: 93.90 -- Train Loss: 0.4322803318500519
Enter epoch 4, learning rate = 0.009999999776482582
Epoch 4 - Train Acc: 94.95 -- Train Loss: 0.3755902945995331
Enter epoch 5, learning rate = 0.009999999776482582
Epoch 5 - Train Acc: 95.10 -- Train Loss: 0.3691486418247223
Enter epoch 6, learning rate = 0.009999999776482582
Epoch 6 - Train Acc: 95.58 -- Train Loss: 0.33545997738838196
Enter epoch 7, learning rate = 0.009999999776482582
Epoch 7 - Train Acc: 95.86 -- Train Loss: 0.3201998770236969
Enter epoch 8, learning rate = 0.009999999776482582
Epoch 8 - Train Acc: 96.13 -- Train Loss: 0.30639490485191345
Enter epoch 9, learning rate = 0.009999999776482582
Epoch 9 - Train Acc: 96.26 -- Train Loss: 0.2964937686920166
Enter epoch 10, learning rate = 0.

In [13]:
test_data_loader = DataLoader("./../ember_data/test_set.pickle")
test_ds = test_data_loader.create_tensorflow_dataset()

for test_images, test_labels in test_ds:
    test_acc, test_loss = tensorflow_framework.evaluate(test_images, test_labels)

print(f"Epoch {epoch+1} - Test Acc: {(test_acc * 100):.2f} -- Test Loss: {test_loss}")


2023-08-13 14:51:53.297033: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


Epoch 10 - Test Acc: 93.19 -- Test Loss: 0.40984147787094116
